Лицензия MIT

© Алексей Александрович Щербаков, 2024

# Лекция 5.3. Глубокое обучение в задачах оптики и фотоники

Глубокое обучение основано на использовании глубоких нейронных сетей (DNN) для решения сложных задач оптимизации. В отличие от традиционных аналитических подходов, глубокое обучение оказывается полезно, когда знаний о рассматриваемой системе недостаточно для применения более традиционных оптимизационных методов, или когда существует слишком много степеней свободы, что существенно затрудняют прогнозирование поведения системы. В таких сценариях «черного ящика» сети можно обучать, используя только входные и выходные данные, без необходимости знания и понимания внутренней работы системы. С одной стороны, такой подход идет несколько в разрез с традиционным научным и инженерным методом, но бывают ситуации, когда только он позволяет получать требуемые решения.